Tekli-Tanıma

In [ ]:
import cv2

# Kamera başlat
cap = cv2.VideoCapture(0)
tracker = cv2.TrackerCSRT_create()

# İlk frame al
ret, frame = cap.read()
if not ret:
    print("Kamera açılamadı.")
    exit()

# Kullanıcıdan nesne seçmesini iste - sectikten sonra enter basmalı
bbox = cv2.selectROI("Nesne Seç", frame, fromCenter=False, showCrosshair=True)
cv2.destroyWindow("Nesne Seç")

# Tracker'ı başlat
tracker.init(frame, bbox)

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)  # Ayna efekti (isteğe bağlı)
    if not ret:
        break

    success, box = tracker.update(frame)

    if success:
        (x, y, w, h) = [int(v) for v in box]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
        print(f"Konum: x={x}, y={y}, w={w}, h={h}")
    else:
        cv2.putText(frame, "Takip kaybedildi", (100,80),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,255),2)

    cv2.imshow("Takip", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == 27:  # ESC
        break

cap.release()
cv2.destroyAllWindows()


Konum: x=163, y=260, w=85, h=67
Konum: x=154, y=333, w=87, h=68
Konum: x=153, y=338, w=85, h=67
Konum: x=152, y=338, w=85, h=67
Konum: x=152, y=339, w=85, h=67
Konum: x=151, y=338, w=85, h=67
Konum: x=150, y=338, w=87, h=68
Konum: x=149, y=338, w=87, h=68
Konum: x=150, y=338, w=87, h=68
Konum: x=149, y=338, w=87, h=68
Konum: x=148, y=337, w=88, h=70
Konum: x=147, y=337, w=90, h=71
Konum: x=146, y=338, w=92, h=73
Konum: x=145, y=339, w=92, h=73
Konum: x=145, y=339, w=92, h=73
Konum: x=144, y=339, w=92, h=73
Konum: x=143, y=339, w=92, h=73
Konum: x=143, y=339, w=92, h=73
Konum: x=143, y=341, w=90, h=71
Konum: x=143, y=340, w=90, h=71
Konum: x=142, y=339, w=90, h=71
Konum: x=140, y=339, w=92, h=73
Konum: x=140, y=339, w=92, h=73
Konum: x=140, y=340, w=92, h=73
Konum: x=140, y=340, w=92, h=73
Konum: x=140, y=340, w=92, h=73
Konum: x=139, y=340, w=92, h=73
Konum: x=138, y=339, w=94, h=74
Konum: x=137, y=340, w=94, h=74
Konum: x=137, y=340, w=92, h=73
Konum: x=136, y=339, w=96, h=75
Konum: x

Çoklu-tanıma , video link :https://pixabay.com/tr/videos/koşma-su-yolu-irlanda-atletizm-22914/

In [ ]:
import cv2

# Desteklenen tracker isimleri
trackers = {
    "MIL": cv2.legacy.TrackerMIL_create,
    "KCF": cv2.legacy.TrackerKCF_create,
    "CSRT": cv2.legacy.TrackerCSRT_create,
    "TLD": cv2.legacy.TrackerTLD_create,
    "MEDIANFLOW": cv2.legacy.TrackerMedianFlow_create,
    "MOSSE": cv2.legacy.TrackerMOSSE_create
}

# Kullanılacak tracker tipi
tracker_type = "CSRT"  # burada değiştirebilirir tracker türü

# Video dosyasını aç
cap = cv2.VideoCapture("kosu.mp4") # video dosya Link :https://pixabay.com/tr/videos/koşma-su-yolu-irlanda-atletizm-22914/

ret, frame = cap.read()
if not ret:
    print("Video okunamadı.")
    cap.release()
    exit()

boxes = []

while True:
    # Kullanıcıdan ROI seçimi - en az bir nesne seçilmeli
    box = cv2.selectROI("Nesne Seç", frame, fromCenter=False, showCrosshair=True)
    if box == (0, 0, 0, 0):
        print("Geçerli bir seçim yapılmadı, tekrar deneyin.")
        continue

    boxes.append(box)
    print("q ile çık, başka nesne eklemek için herhangi bir tuşa bas.")

    if cv2.waitKey(0) & 0xFF == ord("q"):
        break

# MultiTracker başlat
multi_tracker = cv2.legacy.MultiTracker_create()

for box in boxes:
    tracker = trackers[tracker_type]()  # Seçilen tracker türünden nesne oluştur
    multi_tracker.add(tracker, frame, box)

print(f"Seçilen nesne sayısı: {len(boxes)}")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Frame alınamadı, video bitti veya hata oluştu.")
        break

    success, multi_boxes = multi_tracker.update(frame)

    for i, newbox in enumerate(multi_boxes):
        (x, y, w, h) = [int(v) for v in newbox]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, f"Nesne {i+1}", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        

        print(f"Nesne {i+1}: x={x}, y={y}, w={w}, h={h}")

    cv2.imshow("Takip", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


q ile çık, başka nesne eklemek için herhangi bir tuşa bas.
q ile çık, başka nesne eklemek için herhangi bir tuşa bas.
Geçerli bir seçim yapılmadı, tekrar deneyin.
Geçerli bir seçim yapılmadı, tekrar deneyin.
Geçerli bir seçim yapılmadı, tekrar deneyin.
q ile çık, başka nesne eklemek için herhangi bir tuşa bas.
Seçilen nesne sayısı: 3
Nesne 1: x=488, y=139, w=41, h=115
Nesne 2: x=651, y=159, w=24, h=66
Nesne 3: x=712, y=161, w=30, h=71
Nesne 1: x=485, y=136, w=41, h=118
Nesne 2: x=652, y=161, w=23, h=63
Nesne 3: x=712, y=160, w=31, h=72
Nesne 1: x=481, y=135, w=41, h=118
Nesne 2: x=650, y=161, w=23, h=63
Nesne 3: x=711, y=158, w=32, h=75
Nesne 1: x=476, y=133, w=42, h=120
Nesne 2: x=650, y=163, w=22, h=62
Nesne 3: x=710, y=157, w=32, h=77
Nesne 1: x=471, y=134, w=42, h=120
Nesne 2: x=649, y=164, w=22, h=62
Nesne 3: x=708, y=157, w=32, h=77
Nesne 1: x=469, y=137, w=41, h=118
Nesne 2: x=648, y=163, w=22, h=62
Nesne 3: x=707, y=156, w=33, h=78
Nesne 1: x=465, y=140, w=41, h=118
Nesne 2: x=